# L5: Agentic RAG & External Memory


## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

Letta agents persist information over time and restarts by saving data to a database. These lessons do not require past information. To enable a clean restart, the database is cleared before starting the lesson.

In [1]:
!rm  -f ~/.letta/sqlite.db

## Section 0: Setup a client 

In [2]:
from helper import nb_print, load_env
from dotenv import load_dotenv

_ = load_dotenv()

In [3]:
from letta import create_client 

client = create_client()

Saved Config:  /home/jovyan/.letta/config
📖 Letta configuration file updated!
🧠 model	-> gpt-4
🖥️  endpoint	-> http://jupyter-api-proxy.internal.dlai/rev-proxy/letta
Saved Config:  /home/jovyan/.letta/config
Saved Config:  /home/jovyan/.letta/config


In [4]:
from letta.schemas.llm_config import LLMConfig

client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini"))

## Section 1: Loading data into archival memory 

In [5]:
source = client.create_source("employee_handbook")
source

Source(description=None, embedding_config=EmbeddingConfig(embedding_endpoint_type='openai', embedding_endpoint='http://jupyter-api-proxy.internal.dlai/rev-proxy/letta', embedding_model='text-embedding-ada-002', embedding_dim=1536, embedding_chunk_size=300, azure_endpoint=None, azure_version=None, azure_deployment=None), metadata_=None, id='source-78d2b518-e5f7-43b6-a4b2-5d829971109e', name='employee_handbook', created_at=datetime.datetime(2024, 11, 8, 2, 15, 47, 126845, tzinfo=datetime.timezone.utc), user_id='user-93ec88f7-42f9-4987-a7e8-337df3ce8aea')

In [6]:
client.load_file_into_source(
    filename="handbook.pdf", 
    source_id=source.id
)

Loading files: 100%|██████████| 1/1 [00:00<00:00,  2.00file/s]


Job(metadata_={}, id='job-707d66ba-de24-4f73-8ef4-cc75551dd27f', status=<JobStatus.created: 'created'>, created_at=datetime.datetime(2024, 11, 8, 2, 16, 8, 532301, tzinfo=datetime.timezone.utc), completed_at=None, user_id='user-93ec88f7-42f9-4987-a7e8-337df3ce8aea')

In [7]:
agent_state = client.create_agent()

In [8]:
client.attach_source_to_agent(
    agent_id=agent_state.id, 
    source_id=source.id
)

100%|██████████| 1/1 [00:00<00:00, 26.41it/s]


In [9]:
client.list_attached_sources(agent_state.id)

[Source(description=None, embedding_config=EmbeddingConfig(embedding_endpoint_type='openai', embedding_endpoint='http://jupyter-api-proxy.internal.dlai/rev-proxy/letta', embedding_model='text-embedding-ada-002', embedding_dim=1536, embedding_chunk_size=300, azure_endpoint=None, azure_version=None, azure_deployment=None), metadata_=None, id='source-78d2b518-e5f7-43b6-a4b2-5d829971109e', name='employee_handbook', created_at=datetime.datetime(2024, 11, 8, 2, 15, 47, 126845), user_id='user-93ec88f7-42f9-4987-a7e8-337df3ce8aea')]

In [12]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Search archival for our company's vacation policies", 
    role = "user"
) 

nb_print(response.messages)

## Section 2: Connecting data via tools

In [14]:
def query_birthday_db(self, name: str): 
    """
    This tool queries an external database to 
    lookup the birthday of someone given their name.

    Args: 
        name (str): The name to look up 

    Returns: 
        birthday (str): The birthday in mm-dd-yyyy format
    
    
    """
    my_fake_data = {
        "bob": "03-06-1997", 
        "sarah": "03-06-1997"
    } 
    name = name.lower() 
    if name not in my_fake_data: 
        return None
    else: 
        return my_fake_data[name]

In [15]:
birthday_tool = client.create_tool(query_birthday_db)

In [16]:
birthday_tool

Tool(description=None, source_type='python', module=None, user_id='user-93ec88f7-42f9-4987-a7e8-337df3ce8aea', id='tool-1f3d65db-49a7-48fb-8bc9-f3c74fb3259b', name='query_birthday_db', tags=[], source_code='def query_birthday_db(self, name: str): \n    """\n    This tool queries an external database to \n    lookup the birthday of someone given their name.\n\n    Args: \n        name (str): The name to look up \n\n    Returns: \n        birthday (str): The birthday in mm-dd-yyyy format\n\n\n    """\n    my_fake_data = {\n        "bob": "03-06-1997", \n        "sarah": "03-06-1997"\n    } \n    name = name.lower() \n    if name not in my_fake_data: \n        return None\n    else: \n        return my_fake_data[name]\n', json_schema={'name': 'query_birthday_db', 'description': 'This tool queries an external database to ', 'parameters': {'type': 'object', 'properties': {'name': {'type': 'string', 'description': 'The name to look up '}, 'request_heartbeat': {'type': 'boolean', 'description

In [17]:
from letta.schemas.memory import ChatMemory

agent_state = client.create_agent(
    name="birthday_agent", 
    tools=[birthday_tool.name], 
    memory=ChatMemory(
        human="My name is Sarah", 
        persona="You are a agent with access to a birthday_db " \
        + "that you use to lookup information about users' birthdays."
    )
)

In [18]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "When is my birthday?", 
    role = "user"
) 
nb_print(response.messages)

### Loading tools from Langchain

In [19]:
import getpass
import os
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

In [20]:
from langchain_community.tools import TavilySearchResults
from letta.schemas.tool import Tool

search = TavilySearchResults()

In [21]:
search.run("What's Obama's first name?") 

[{'url': 'https://www.bbc.com/news/world-us-canada-13221643',
  'content': 'Nothing but rubble: Ukraine\'s shattered ghost town Avdiivka\nSecret calls and code names: How money makes it to N Korea\nCounting the destruction of religious sites in Gaza\nLily Gladstone: The actress who could make Oscars history\nGuardiola, Mourinho and the game that changed everything\nWhy India wants to fence its troubled Myanmar border\n\'We\'re the country of beef, but we can only afford chicken\'\nKenya\'s visa-free dream proves tricky for some\nElsewhere on the BBC\nThe truth about burnout\nWhy \'living retro\' is perfect for now\nA 75km hike through \'the Graveyard of the Pacific\'\nMost Read\nBBC News Services\n© 2024 BBC. "The designation of Sr or Jr to distinguish between father and son with all the exact same names (first, middle, & last), can be replaced by the Roman numerals, I and II, respectively, when the grandson has the exact same names," explain Dr Dave and Dr Dee, who provide advice on h

In [22]:
# convert the tool to Letta Tool 
search_tool = Tool.from_langchain(TavilySearchResults())

In [23]:
# persist the tool 
client.add_tool(search_tool)

Tool(description=None, source_type='python', module=None, user_id='user-93ec88f7-42f9-4987-a7e8-337df3ce8aea', id='tool-66426869-5584-4578-b839-f53dcce54dae', name='run_tavilysearchresults', tags=['langchain'], source_code="\ndef run_tavilysearchresults(**kwargs):\n    if 'self' in kwargs:\n        del kwargs['self']\n    from langchain_community.tools import TavilySearchResults\n    tool = TavilySearchResults()\n    return tool._run(**kwargs)\n", json_schema={'name': 'run_tavilysearchresults', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'search query to look up'}, 'request_heartbeat': {'type': 'boolean', 'description': "Request an immediate heartbeat after function execution. Set to 'true' if you want to send a follow-up message or run a follow-up f

In [24]:
research_agent_persona = f"""
You have access to a web via a {search_tool.name} tool. 
Use this tool to respond to users' questions, by summarizing the
{search_tool.name} 
and also providing the `url` that the information was from as a 
reference. 

<Example> 
User: 'What is Obama's first name?' 
Assistant: 'Obama's first name is Barack.

Sources:
[1] https://www.britannica.com/biography/Barack-Obama
[2] https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States'
</Example>
Your MUST provide URLs that you used to generate the answer, or you will be terminated. 

"""

In [25]:
agent_state = client.create_agent(
    name="research_agent", 
    tools=[search_tool.name], 
    memory=ChatMemory(
        human="My name is Sarah", 
        persona=research_agent_persona
    )
)

In [26]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Who founded OpenAI? ", 
    role = "user"
) 
nb_print(response.messages)

In [28]:
from letta.schemas.llm_config import LLMConfig

agent_state = client.create_agent(
    name="gpt4_search_agent", 
    tools=[search_tool.name], 
    memory=ChatMemory(
        human="My name is Sarah", 
        persona=research_agent_persona
    ),
    #llm_config=LLMConfig.default_config('gpt-4')  # uncomment if you are not getting desired results. Note that this is much more expensive than gpt-4o-mini
    llm_config=LLMConfig.default_config('gpt-4o-mini')
)

ValueError: Agent with name gpt4_search_agent already exists (user_id=user-93ec88f7-42f9-4987-a7e8-337df3ce8aea)

In [29]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Who founded OpenAI? ", 
    role = "user"
) 
nb_print(response.messages)